# Deployment

In order to deploy the model 2 steps have to be performed:
- Dump the required data into a CSV the REST service can read
- Start the REST service

### Step 1 - Dump the required data into a CSV

And place it in your service's folder in a **/model/ subfolder**, along with the original items information **product_master.csv**

In [1]:
import pandas as pd

clean_data: pd.DataFrame = pd.read_csv('data_clean_recommendation.csv')
    
grouped_data: pd.DataFrame = clean_data.drop(['month_code', 'log_date', 'is_peak_period'], axis = 1)
grouped_data = grouped_data.groupby(grouped_data.columns.drop('volume_primary_units').tolist()).sum().reset_index()
grouped_data.volume_primary_units.loc[grouped_data.volume_primary_units == 0] = 1
grouped_data = grouped_data[grouped_data.volume_primary_units > 0]
grouped_data.drop('profit_bin', axis = 1);

grouped_data[['product_code', 'cac', 'volume_primary_units', 'country_code', 'cost_bin']].to_csv('base_data.csv')

C:\Users\Horus\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Step 2 - Start the REST service

Run the following command in a local CMD

cd d:/your_app_dir/

uvicorn main:app --reload --workers 1 --host 127.0.0.1 --port 8008

The service is now live at 
http://http://127.0.0.1:8008/

And the automated documentation and live testing can be performed at
http://127.0.0.1:8008/docs

For instance, select the route /predict, and input the following:
{
  "user_id": 12
}

which will generate product recommendations for user_id 12.

Note: this process will take a long to run, as the data will be re-read on each call. In practice, a Singleton has to be created to hold the data or a celery task should be used in a NoSQL context.

The output should return:

Response body:

{
  "user_id": "12",
  "recommended_products": [
    "product_code_1124",
    "product_code_1008",
    "product_code_12451",
    "product_code_1049",
    "product_code_1060",
    "product_code_1230",
    "product_code_1280",
    "product_code_12710",
    "product_code_11568",
    "product_code_12393"
  ],
  "known_products": []
}

Response headers:

 content-length: 265 
 content-type: application/json 
 date: Tue,08 Dec 2020 22:30:09 GMT 
 server: uvicorn 